In [ ]:
from qat.lang.AQASM import Program, QRoutine, H, CNOT, RX, RY, RZ, X, Z
from qat.qpus import get_default_qpu
from qat.core import Observable, Term
from qat.plugins import ObservableSplitter, ScipyMinimizePlugin

import numpy as np
import random
# pyqubo imports
from pyqubo import Binary, Spin, Array
from pprint import pprint
import matplotlib.pyplot as plt

# Hands-on: Problema da Mochila no VQE

## Descrição do problema

Imagine que voce tem uma mochila com capacidade $W$ (peso total) e barras de ouro de diversos pesos, $w_i$. Você quer colocar o máximo de ouro dentro da mochila, sem que sua capacidade seja ultrapassada. Quais barras de ouro colocar de forma a maximizar a quantidade de ouro dentro da mochila? Esta é uma versão simplificada do problema. Este exemplo pode ser generalizado para várias mochilas e objetos com valores e pesos diferentes.

<img src="imagens/mochila.jpeg" width=750 />

## Formulação como otimização irrestrita

Como queremos maximizar a quantidade de ouro dentro da mochila, podemos criar uma função objetivo de minimização da seguinte forma

$$H(\vec{x}) = \left(W - \sum_{i=0}^{n}w_ix_i\right)^2$$
onde $x_i=1$ se a barra de ouro $i$ está dentro da mochila e $x_i=0$ caso contrário. 





## Hands-on

Vamos fazer um exemplo simples, com poucas variáveis. Imagine que você possui 3 Barras: Barra 1 - `5kg`; Barra 2 - `3kg`; e Barra 3 - `1kg`. No entando, sua mochila tem a capacidade de apenas `6kg`. Qual é a melhor combinação de barras a se guardar na mochila de forma a minimizar a sobra? Use o VQE para solucionar o problema.

In [ ]:
num_variables = 3
variables = Array.create('x', shape=num_variables, vartype='BINARY')
weights = [5,3,1]

### Exercício 1

Crie a função objetivo do problema usando o `Pyqubo`.

In [ ]:
H = None

H

### Converção para modelo de Ising

In [ ]:
# -----------------------------------
# Transformando de Binary para Ising
# -----------------------------------

model = H.compile()
model = model.to_ising()
linear, quadratic = model[0], model[1]

print("Termos lineares do modelo de Ising", linear)
print("Termos quadráticos do modelo de Ising", quadratic)

### Exercício 2

Escreva o modelo de Ising em termos de operadores de Pauli-Z. Crie um dicionário para armazenar cada termo da seguinte forma:

Exemplo: `hamiltonian = {"ZIZ": 4, ...}` onde as chaves são strings com operadores de Pauli-Z e Identidade e os respectivos valores são seus coeficientes.

Para resolvermos o nosso problema usando o myQLM criaremos um observável usando o Hamiltoniano de Ising criado acima.

In [ ]:
single_values = {}
multiple_values = {}
for k,v in linear.items():
    single_values[int(k[2])] = v
for k, v in quadratic.items():
    multiple_values[(int(k[0][2]), int(k[1][2]))] = v

num_qubits = len(variables)
hamiltonian = Observable(num_qubits
                           , pauli_terms=
                           [Term(single_values[x], "Z", [x]) for x in single_values]+
                           [Term(multiple_values[x], "ZZ", [x[0],x[1]]) for x in multiple_values]
                           )

print(hamiltonian)

### Ansatz

O ansatz é um circuito quântico parametrizado. Sua forma variacional pode ser dada por padrões heurísticos geralmente implementada através de operadores de rotação e portas CNOT para realizar emaranhamento entre os qubits. Uma forma variacional eficiente é aquela capaz de generalizar bem um estado quântico, aumentando o espaço de busca. Podemos escrever a atuação do circuito variacional, $U(\vec{\theta})$, sobre um sistema com $n$ qubits de estado inicial, $|0\rangle^{\otimes n}$, como

$$U(\vec{\theta})|0\rangle^{\otimes n} = |\psi(\vec{\theta})\rangle$$

Agora, realizaremos o mesmo processo do exemplo anterios, mas agora não estaremos interessados em ter um número como solução, e sim um vetor, o $\vec{z}$. Embora minimizaremos o valor esperado de um operador, estaremos interessados no autoestado com maior probabilidade, associado ao menor valor esperado.

### Exercício 3

Aplique as operações no ansatz e defina, no lugar do `FIXME`, o numero de parametros variacionais que voce usou.

In [ ]:
def ansatz(num_qubits, var_params):

    routine = QRoutine()
    wires = routine.new_wires(num_qubits)

    # Escreva seu código aqui
    
    return routine

num_params = 2*num_variables
circuit = ansatz(num_variables, [i for i in range(num_params)])
circuit.display()

In [ ]:
qprog = Program()
qbits = qprog.qalloc(num_variables)

num_layers = 3
num_params = 2*num_variables
params  = [[qprog.new_var(float, '\\param%s'%i) for i in range(num_params)] for j in range(num_layers)]

for i in range(num_layers):
    qprog.apply(ansatz(num_variables, params[i]), qbits)


### Visualização do Circuito

In [ ]:
circuit = qprog.to_circ()
print("total number of gates: ", len(circuit.ops))
print("Variables:", circuit.get_variables())
# Display quantum circuit
%qatdisplay circuit --svg

## Valor esperado do operador hamiltoniano

O valor esperado pode ser dado pelo valor médio das energias. O valor esperado é a função a ser minimizada e pode ser dado por

$$\langle \psi_\theta | H | \psi_\theta \rangle = \sum_{i} E_ip_i$$

### Exercício 4


In [ ]:
# Create a job
job = circuit.to_job(observable=hamiltonian)

result_list = []
# Escolha um número de iterações adequado
num_iterations = FIXME
for _ in range(num_iterations):
    # Inicialize os parâmetros iniciais
    initial_parameters = FIXME
    cobyla = ScipyMinimizePlugin(tol=1e-6,
                                method="COBYLA",
                                options={"maxiter": 300},
                                x0=initial_parameters)
    # Create a Quantum Processor Unit
    qpu = get_default_qpu()

    stack =  cobyla | qpu

    # Submit the job to the QPU
    result_list.append(stack.submit(job))

#### Análise dos Resultados

In [ ]:
for i, r in enumerate(result_list):
    print("Run", i, ", Final energy:", r.value)
    #Binding the variables:random.uniform(0, 2*np.pi)
    sol_job = job(**eval(r.meta_data["parameter_map"]))

    #Rerunning in 'SAMPLE' mode to get the most probable states:
    sampling_job = sol_job.circuit.to_job()

    sol_res = qpu.submit(sampling_job)
    print("Most probable states are:")
    for sample in sol_res:
        if sample.probability > 0.05:
            print(sample.state, "{:.2f}%".format(100 * sample.probability))

In [ ]:
# Escolhendo o resultado com o menor valor esperado
result = min(result_list, key=lambda s: s.value)
print("Final energy:", result.value)
for key, value in result.meta_data.items():
    print(key, ":", value)

In [ ]:
plt.plot(eval(result.meta_data["optimization_trace"]))
plt.xlabel("steps")
plt.ylabel("energy")
plt.show()

### Exercicio 5

Pegue os melhores parametros encontrados na minimização do valor esperado e aplique-os no ansatz. Este circuito será o circuito "ótimo" e medidas neste circuito nos dará informações sobre as melhores soluções.

In [ ]:
# Emulating a reasonable setup:
# Drawing 2048 cuts
sol_job = job(**eval(result.meta_data["parameter_map"]))
sampling_job = sol_job.circuit.to_job(nbshots=2048)
sol_res = qpu.submit(sampling_job)

max_state = max([(s.state.value[0], s.probability) for s in sol_res], key=lambda s: s[1])
print("State with highest probability:"
      , max_state[0]
      , "%.2f%%" % (100 * max_state[1]) )



O autoestado com maior ocorrência indica a solução ótima.